In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn

In [2]:
df = pd.read_parquet('./K-means/new_changes_done.parquet')

In [3]:
df = df.loc[df['report_date']=='2020-02-29']
#df = df.loc[df['report_date']=='2020-01-31']
df = df.loc[df['fund_id']=='04BBSH-E']
df = df.iloc[:,4:]
df.set_index(['stock_id'],inplace=True)
df = df.fillna(df.median())
target = df.pop('is_holding')
df.insert(9,'is_holding',target) 
df

,roe,roa,oper_mgn,pay_out_ratio,pe,pbps,div_yld,last_holding,new_changes,is_holding
stock_id,,,,,,,,,,
T1N9J9-S-RPMZQ3-R-04BBSH-E,0.736257,0.528786,0.895426,0.751899,0.270954,0.389738,0.697859,0.0,0.0,0
DTMZTR-S-NMTXM7-R-04BBSH-E,0.000000,0.086202,0.118942,0.307280,1.000000,1.000000,0.000000,0.0,0.0,0
R5VL9T-S-KJZM9C-R-04BBSH-E,0.181851,0.277281,0.251268,0.307280,1.000000,1.000000,0.222079,0.0,0.0,0
G18BJK-S-L2LQRT-R-04BBSH-E,0.994465,0.969054,0.910720,0.922097,0.157442,1.000000,0.974442,0.0,0.0,0
SVR7X0-S-F0M3M6-R-04BBSH-E,0.374042,0.383676,0.626002,0.307280,0.757425,0.779166,0.222079,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
B2TSYX-S-VS1SJK-R-04BBSH-E,0.120505,0.068349,0.000000,0.307280,1.000000,1.000000,0.000000,0.0,0.0,0
KHYJ27-S-VDP7J2-R-04BBSH-E,0.495329,0.728894,0.382177,0.970322,0.865741,1.000000,0.000000,0.0,0.0,0
QQ0TFB-S-GB4QMW-R-04BBSH-E,0.384518,0.399007,0.641596,0.307280,0.716920,0.750751,0.000000,0.0,0.0,0


In [4]:
### save the column and index
columns = df.columns
index = df.index.values

In [5]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')

In [6]:
### The following function is a recommendation system that the user can import the data, choose one specific bond, and select
### some features, then the system will return k bonds most similar to the chosen bond based on the selected features.

def stock_recommend(dataframe,stock,k):
    
    # import the dataframe and select the features
    # df = dataframe[self_selected_features_list]
    
    # create a dictionary for bond names
    stocks = df.index.values
    d = {}
    for i in range(len(stocks)):
        d[stocks[i]] = i
    
    # find the indices for the choosing bond
    idx = d[stock]
    
    # create a list of most similar k bonds
    knn.fit(df)
    distances, indices = knn.kneighbors(df.values, n_neighbors = k+1)
    sim_stocks_idx = indices[idx].tolist()
    
    # remove bond_A from the list sim_bonds
    sim_stocks_idx.remove(idx)
    sim_stocks = df.index[sim_stocks_idx].to_list()
    
    print('Most similar stock to stock: {} is: {}'.format(stock, sim_stocks))

In [7]:
stock_recommend(df,'T1N9J9-S-RPMZQ3-R-04BBSH-E', 1)

Most similar stock to stock: T1N9J9-S-RPMZQ3-R-04BBSH-E is: ['WW3RWY-S-Q1LLR2-R-04BBSH-E']


In [8]:
stock_recommend(df,'T1N9J9-S-RPMZQ3-R-04BBSH-E', 3)

Most similar stock to stock: T1N9J9-S-RPMZQ3-R-04BBSH-E is: ['WW3RWY-S-Q1LLR2-R-04BBSH-E', 'BW90JZ-S-SC41Y3-R-04BBSH-E', 'FBBGRD-S-Q8DBNV-R-04BBSH-E']


In [8]:
stock_recommend(df,'WW3RWY-S-Q1LLR2-R-04BBSH-E', 1)

Most similar stock to stock: WW3RWY-S-Q1LLR2-R-04BBSH-E is: ['T1N9J9-S-RPMZQ3-R-04BBSH-E']


In [9]:
stock_recommend(df,'T1N9J9-S-RPMZQ3-R-04BBSH-E', 2)

Most similar stock to stock: T1N9J9-S-RPMZQ3-R-04BBSH-E is: ['WW3RWY-S-Q1LLR2-R-04BBSH-E', 'BW90JZ-S-SC41Y3-R-04BBSH-E']
